# Running multiple calculations on a given model


## Aim

This notebook shows how we can run multiple calculations of a given structure

### Setup

The initial setup is very similar to the other tutorials, such as `singlepoint.ipynb`, which goes into more detail about what each step is doing

Load the aiida profile and code:

In [1]:
from aiida import load_profile
load_profile()

Profile<uuid='fcd6e413ff67451981fc4de1bfe379bf' name='scarf'>

In [2]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [3]:
from aiida.orm import load_code
janus_code = load_code("janus@localhost")

Inputs should include the model, code, metadata, and any other keyword arguments expected by the calculation we are running:

In [4]:
from aiida.orm import Str, Float, Bool
inputs = {
    "code": janus_code,
    "model": model,
    "arch": Str(model.architecture),
    "precision": Str("float64"),
    "device": Str("cpu"),
    "fmax": Float(0.1), 
    "opt_cell_lengths": Bool(False), 
    "opt_cell_fully": Bool(True), 
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    }

We must now choose the calculations to perform:

In [5]:
from aiida.plugins import CalculationFactory
geomoptCalc = CalculationFactory("mlip.opt")
descriptorsCalc = CalculationFactory("mlip.descriptors")


Now we can create our WorkGraph. This includes passing in the inputs, checking the amount of structures we have and interating through. Note for this workbook we have decreased the amount of structures to two for the sake of simplicity. In the loop we can call each structure; run geomopt calculation, pass the output file into descriptors calculation, and get the final outputs of all the structures

In [6]:
from sample_split import build_filters_workgraph
from aiida.orm import Str, Float, Bool, Int
from ase.io import read

structure = "../../examples/tutorials/structures/lj-traj.xyz"

split_inputs = {
    "config_types": Str(""),
    "prefix": Str(""),
    "scale": Float(1.0e5),
    "append_mode": Bool(False)
}

wg = build_filters_workgraph(
    initial_struct = structure,
    calc_inputs = [geomoptCalc,descriptorsCalc],
    wg_inputs= inputs,
    split_inputs = split_inputs
)



defining outputnode
defining outputnode


In [7]:
wg


NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'Calcula…

In [8]:
wg.run()

08/13/2025 09:31:06 AM <85934> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
08/13/2025 09:31:06 AM <85934> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
08/13/2025 09:31:06 AM <85934> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.
08/13/2025 09:31:23 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [688|WorkGraphEngine|continue_workgraph]: tasks ready to run: GeomOpt,GeomOpt1,GeomOpt2,GeomOpt3,GeomOpt4,GeomOpt5,GeomOpt6,GeomOpt7,GeomOpt8,GeomOpt9,GeomOpt10
08/13/2025 09:31:45 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [688|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 692, 696, 700, 704, 708, 712, 716, 720, 724, 728, 732
08/13/2025 09:32:48 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkC

create files: train_file=PosixPath('train.xyz'), valid_file=PosixPath('valid.xyz') and test_file=PosixPath('test.xyz')
Processing: ('all', 'aiida'), 11 frames
  ('all', 'aiida'): total=11, train_target=8,                     vt_target=3


/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 11
  warnings.warn(f"k is too large, set to {n_pts}")
/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 3
  warnings.warn(f"k is too large, set to {n_pts}")
08/13/2025 09:35:49 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [688|WorkGraphEngine|update_task_state]: Task: process_and_split_data, type: CALCFUNCTION, finished.
08/13/2025 09:35:50 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [688|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
08/13/2025 09:35:54 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [688|WorkGraphEngine|finalize]: Finalize workgraph.


08/13/2025 10:17:44 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1514|WorkGraphEngine|update_task_state]: Task: PwCalculation2, type: CALCJOB, finished.
08/13/2025 10:18:57 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1514|WorkGraphEngine|update_task_state]: Task: PwCalculation, type: CALCJOB, finished.
08/13/2025 10:18:58 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1514|WorkGraphEngine|update_task_state]: Task: PwCalculation1, type: CALCJOB, finished.
08/13/2025 10:19:30 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1514|WorkGraphEngine|update_task_state]: Task: PwCalculation3, type: CALCJOB, finished.
08/13/2025 10:19:31 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1514|WorkGraphEngine|update_task_state]: Task: PwCalculation5, type: CALCJOB, finished.
08/13/2025 10:20:03 AM <85934> aiida.orm.nodes.process.workflow.wo

In [10]:
from ase.io import read

file_dict = wg.tasks.process_and_split_data.outputs.result.value.get_dict()
train = file_dict['train_file']

print(len(read(train, index=':')))

8


Now we can run the calculations

In [11]:
from aiida.orm import load_node, load_code, load_computer
from aiida import load_profile


load_profile("scarf", allow_switch=True)
qe_code = load_code("qe@scarf")

computer_label = 'scarf'
scarf = load_computer(computer_label)
scarf.set_workdir('/work4/scd/scarf1480/aiida') 
# file_dict = load_node(3588).get_dict()

In [12]:
from ase.build import bulk
from aiida.orm import Dict, KpointsData, StructureData, load_code, load_group
from aiida.plugins.factories import WorkflowFactory

from ase.io import read
from aiida_workgraph import WorkGraph
from aiida_quantumespresso.calculations.pw import PwCalculation

kpoints = KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
pseudo_family = load_group('SSSP/1.3/PBE/efficiency')
metadata =  {
    "options": {
        "resources": {
            "num_machines": 1
        },
        'max_wallclock_seconds': 3600,         # Set maximum wallclock time
        #'account': 'elph',                     # Set account name
        'queue_name': 'scarf',                  # Set queue name
        'qos': 'scarf',
        'environment_variables': {},
        'withmpi': True,                       # Use MPI
        'prepend_text': '''
        module purge
        module use /work4/scd/scarf562/eb-common/modules/all
        module load amd-modules
        module load QuantumESPRESSO/7.2-foss-2023a
        ''',
        'append_text': ''  
    }
}

output_traj = {}
with WorkGraph("QE") as wg:

    for _, filepath_out in file_dict.items():

        for i in range(len(read(filepath_out, index=':'))):

            structure = StructureData(ase=read(filepath_out, index=i))
            pseudos = pseudo_family.get_pseudos(structure=structure)

            ecutwfc, ecutrho = pseudo_family.get_recommended_cutoffs(
                structure=structure,
                unit='Ry',
            )

            pw_paras = {
                "CONTROL": {
                    "calculation": "scf",
                },
                "SYSTEM": {
                    "ecutwfc": ecutwfc,
                    "ecutrho": ecutrho,
                },
            }

            qe_task = wg.add_task(
                PwCalculation,
                code= qe_code,
                parameters= pw_paras,
                kpoints= kpoints,
                pseudos= pseudos,
                metadata= metadata,
                structure= structure,
            )

            output_traj[f"{_}.struct{i}"] = qe_task.outputs.output_trajectory

wg.outputs = output_traj
    



In [13]:
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'QE', 'u…

In [ ]:
wg.outputs.train_file

In [ ]:
wg.run()

08/13/2025 10:09:10 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1514|WorkGraphEngine|continue_workgraph]: tasks ready to run: PwCalculation,PwCalculation1,PwCalculation2,PwCalculation3,PwCalculation4,PwCalculation5,PwCalculation6,PwCalculation7,PwCalculation8,PwCalculation9,PwCalculation10
08/13/2025 10:09:22 AM <85934> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1514|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 1516, 1518, 1520, 1522, 1524, 1526, 1528, 1530, 1532, 1534, 1536
/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/aiida/transports/plugins/ssh.py:674: AiidaDeprecationWarning: `chdir()` is deprecated and will be removed in the next major version. Use absolute paths instead. (this will be removed in v3)
  warn_deprecation(
/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/aiida/transports/plugins/ssh.py:674: AiidaDeprecationWarning: `chdir()` is deprecated and will be remo

In [ ]:
from aiida.orm import load_node
qe_output = load_node(1514).outputs
qe_test_output = load_node(1514).outputs.test_file
qe_train_output = load_node(1514).outputs.train_file
qe_valid_output = load_node(1514).outputs.valid_file

print(qe_test_output.items())
print(qe_output)
# print(qe_output.get_step_structure(0).get_ase())

dict_items([('struct0', <TrajectoryData: uuid: 6f125cde-d06d-443c-be00-bc692d157498 (pk: 1555)>), ('struct1', <TrajectoryData: uuid: 3509cf5d-505f-4b77-a4da-872bed59ad47 (pk: 1558)>)])
Manager for outgoing RETURN links for node pk=1514


In [68]:
TrainCalculation = CalculationFactory("mlip.train")